# Off Ingredients Analysis

Background: All products with ingredient image have automatic extraction. Products without image and manual data input are a lot less trustworthy.

## Data Extraction

### jsonl.gz download + jq

`code,images,has_analysis_tags,ingredients_percent_analysis,ingredients_n,known_ingredients_n,unknown_ingredients_n,states,quality_info`

```
time (
  echo "code,images,ingredients_percent_analysis,ingredients_n,known_ingredients_n,unknown_ingredients_n,states,quality_info"
  zcat < openfoodfacts-products.jsonl.gz | jq -r '[.code, (if .images != null then [.images | to_entries[] | select(.key | startswith("ingred")) | .key] | join(",") else null end), (if .ingredients_analysis_tags != null then true else false end), .ingredients_percent_analysis, .ingredients_n, .known_ingredients_n, .unknown_ingredients_n, (.states_tags // [] | map(select(startswith("en:ing"))) | join(",")), (.data_quality_info_tags // [] | map(select(startswith("en:ing"))) | join(","))] | @csv'
) >ingredients_data_export_jq.csv
```

### ~With DuckDB~ - ***WARNING: DUCKDB has issues!***

- use `array_to_string` instead of `to_json` for serializing lists.
- use `DISTINCT` to avoid duplicates introduced by duckdb for some reason.
- use `FORCE_QUOTE` to make it compatible with jq output

```
COPY (
    SELECT DISTINCT
        code,
        array_to_string(list_filter(json_keys(images), x -> x LIKE 'ingred%'), ',') AS images,
        ingredients_analysis_tags IS NOT NULL AS has_analysis_tags,
        ingredients_percent_analysis,
        ingredients_n,
        known_ingredients_n,
        unknown_ingredients_n,
        array_to_string(list_filter(states_tags, x -> x LIKE 'en:ing%'), ',') as states,
        array_to_string(list_filter(data_quality_info_tags, x -> x LIKE 'en:ing%'), ',') as quality_info
    FROM read_ndjson('openfoodfacts-products.jsonl.gz', ignore_errors=True)
) TO 'ingredients_data_export_dd.csv' (FORMAT CSV, HEADER, FORCE_QUOTE (code, images, states, quality_info));
```

### Compare JQ with DuckDB output

`comm -2 -3 ingredients_data_export_jq.csv ingredients_data_export_dd.csv`

## JSONL export issues
- Malformed NDJSON: unexpected character -> issues for Duckdb
- ~Dataset has 20.756 duplicate rows (YDATA: 74.293 (2.0%))~
- `ingredients_n`, `known_ingredients_n`, and `unknown_ingredients_n` have mixed data types: integer, float, string, e.g. 1.0, 1, "1" and NAN

## Data Completeness
- `ingredients_n` has 2.631.936 (71.3%) missing values
- `images.ingredients` has 2.672.290 (72.4%) missing values

## Data Findings
- both states `en:ingredients-completed` + `en:ingredients-photo-selected` only for 588.881 (16%)
- `known_ingredients_n` has 71.038 (1.9%) zeros
- `known_ingredients_n + unknown_ingredients_n != ingredients_n` for 959.383 (90.5%)
- `unknown_ingredients_n == 0` for 381.903 (10.3%)

## high quality criteria?
- `images.ingredients` and `unknown_ingredients_n == 0` for 228.534 (6.2%)

In [1]:
import pandas as pd
import numpy as np
import ast

In [2]:
!wc -l ../data/ingredients_data_export.csv

 3691922 ../data/ingredients_data_export.csv


In [3]:
!grep -E -v '^[0-9]' ../data/ingredients_data_export.csv

code,images,has_analysis_tags,ingredients_percent_analysis,ingredients_n,known_ingredients_n,unknown_ingredients_n,states,quality_info
"",,false,,,,,"[""en:ingredients-to-be-completed""]",
"",[],false,,,,,"[""en:ingredients-to-be-completed"",""en:ingredients-photo-to-be-selected""]",


In [4]:
%%time
file_path = "../data/ingredients_data_export.csv"
df = pd.read_csv(file_path, dtype={'code':str, 'ingredients_n': str})
df = df[df.code.notna()] # exclude empty barcodes

# extract integer part of int, float, and str (also works for NAN)
def normalize_int(s: pd.Series):
    return s.astype(str).str.extract(r'(\d+)').astype('Int64')

def remove_empty_lists(s: pd.Series):
    return s.map(lambda x: np.nan if x =='[]' else x) # else ast.literal_eval(x)) # or x==np.nan) else ast.literal_eval(x))

for col in ['ingredients_n', 'known_ingredients_n', 'unknown_ingredients_n']:
    df[col] = normalize_int(df[col])

for col in ['images', 'states', 'quality_info']:
    df[col] = remove_empty_lists(df[col])

display(df)

,code,images,has_analysis_tags,ingredients_percent_analysis,ingredients_n,known_ingredients_n,unknown_ingredients_n,states,quality_info
0,0000101209159,"[""ingredients_fr""]",False,NaN,<NA>,<NA>,<NA>,"[""en:ingredients-to-be-completed"",""en:ingredie...",NaN
1,0000105000011,NaN,True,1.0,1,3,0,"[""en:ingredients-completed""]","[""en:ingredients-percent-analysis-ok""]"
2,0000105000042,NaN,True,1.0,1,3,0,"[""en:ingredients-completed""]","[""en:ingredients-percent-analysis-ok""]"
3,0000105000059,NaN,True,1.0,1,0,1,"[""en:ingredients-completed""]","[""en:ingredients-percent-analysis-ok""]"
4,0000105000073,NaN,True,1.0,1,2,0,"[""en:ingredients-completed""]","[""en:ingredients-percent-analysis-ok""]"
...,...,...,...,...,...,...,...,...,...
3691916,8854545889857,NaN,False,NaN,<NA>,<NA>,<NA>,"[""en:ingredients-to-be-completed"",""en:ingredie...",NaN
3691917,8854545889840,NaN,False,NaN,<NA>,<NA>,<NA>,"[""en:ingredients-to-be-completed"",""en:ingredie...",NaN
3691918,8854545889925,NaN,False,NaN,<NA>,<NA>,<NA>,"[""en:ingredients-to-be-completed"",""en:ingredie...",NaN
3691919,8854545889871,NaN,False,NaN,<NA>,<NA>,<NA>,"[""en:ingredients-to-be-completed"",""en:ingredie...",NaN


CPU times: user 9.07 s, sys: 624 ms, total: 9.69 s
Wall time: 9.81 s


# Exploratory Data Analysis

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3691919 entries, 0 to 3691920
Data columns (total 9 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   code                          object 
 1   images                        object 
 2   has_analysis_tags             bool   
 3   ingredients_percent_analysis  float64
 4   ingredients_n                 Int64  
 5   known_ingredients_n           Int64  
 6   unknown_ingredients_n         Int64  
 7   states                        object 
 8   quality_info                  object 
dtypes: Int64(3), bool(1), float64(1), object(4)
memory usage: 267.6+ MB


In [6]:
df.describe(include='all')

,code,images,has_analysis_tags,ingredients_percent_analysis,ingredients_n,known_ingredients_n,unknown_ingredients_n,states,quality_info
count,3691919,1019629,3691919,1.060968e+06,1059983.0,1059869.0,1060600.0,3691919,976647
unique,3671163,5477,2,NaN,<NA>,<NA>,<NA>,6,1
top,0021000007455,"[""ingredients_fr""]",False,NaN,<NA>,<NA>,<NA>,"[""en:ingredients-to-be-completed"",""en:ingredie...","[""en:ingredients-percent-analysis-ok""]"
freq,2,490960,2567706,NaN,<NA>,<NA>,<NA>,1799445,976647
mean,NaN,NaN,NaN,8.467041e-01,15.489712,21.686368,2.965255,NaN,NaN
std,NaN,NaN,NaN,5.320642e-01,16.112839,19.1406,7.025714,NaN,NaN
min,NaN,NaN,NaN,-1.000000e+00,0.0,0.0,0.0,NaN,NaN
25%,NaN,NaN,NaN,1.000000e+00,5.0,7.0,0.0,NaN,NaN
50%,NaN,NaN,NaN,1.000000e+00,11.0,17.0,1.0,NaN,NaN
75%,NaN,NaN,NaN,1.000000e+00,21.0,32.0,3.0,NaN,NaN


## Duplicate Rows

In [7]:
df.code.value_counts()[lambda x: x > 1]

code
0021000007455    2
0016741331532    2
0016767513226    2
0016767122039    2
0016767110609    2
                ..
0011161022473    2
0011161022527    2
0011161022503    2
0011161022510    2
0011161022480    2
Name: count, Length: 20756, dtype: int64

## EDA

In [8]:
!pip install ydata-profiling -q


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [9]:
from ydata_profiling import ProfileReport

profile = ProfileReport(df, explorative=True, minimal=True)
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

## Analyzing ingredients

In [10]:
len(df)

3691919

In [11]:
len(df.unknown_ingredients_n[lambda x: x.notna()])

1060600

In [12]:
len(df.unknown_ingredients_n[lambda x: x == 0])

381903

In [13]:
len(df.unknown_ingredients_n[lambda x: x == 0]) / len(df)

0.10344295202576222

In [14]:
len(df.unknown_ingredients_n[lambda x: x > 0])

678697

In [15]:
match = df.known_ingredients_n + df.unknown_ingredients_n == df.ingredients_n
print(match.value_counts(dropna=False))
print(match.value_counts(dropna=False) / len(df))
print(match.value_counts(dropna=True) / len(df.ingredients_n[df.ingredients_n.notna()]))

<NA>     2632050
False     959383
True      100486
Name: count, dtype: Int64
<NA>     0.712922
False     0.25986
True     0.027218
Name: count, dtype: Float64
False    0.905093
True       0.0948
Name: count, dtype: Float64


In [16]:
len(df[lambda x: (df.images.notna() & (df.unknown_ingredients_n == 0))]) / len(df)

0.06190114138473785

## Analyzing problem with data types

Do this if `*ingredients_n` columns are not normalized.

In [17]:
# test
df.unknown_ingredients_n.map(lambda x: type(x)).value_counts(dropna=False)

unknown_ingredients_n
<class 'pandas._libs.missing.NAType'>    2631319
<class 'int'>                            1060600
Name: count, dtype: int64

In [18]:
# str
df.unknown_ingredients_n[lambda x: x.apply(lambda x: isinstance(x, str))].value_counts(dropna=False)

unknown_ingredients_n
<NA>    0
Name: count, dtype: Int64

In [19]:
# float
df.unknown_ingredients_n[lambda x: x.apply(lambda x: isinstance(x, float))].value_counts(dropna=False)

unknown_ingredients_n
<NA>    0
Name: count, dtype: Int64

In [20]:
# int (after conversion)
df.unknown_ingredients_n[lambda x: x.apply(lambda x: isinstance(x, int))].value_counts(dropna=False)

unknown_ingredients_n
0       381903
1       245219
2       130869
3        79207
4        50697
         ...  
690          1
249          1
153          1
254          1
<NA>         0
Name: count, Length: 225, dtype: Int64

In [21]:
# sanity check: are all floats x.0? -> YES
df.ingredients_n.astype(str).str.extract(r'\.(\d+)').astype('Int64').value_counts(dropna=False)

<NA>    3691919
Name: count, dtype: int64

In [22]:
# extract integer part of int and float (also works with NAN)
def normalize_int(s: pd.Series):
    return s.astype(str).str.extract(r'(\d+)').astype('Int64')

normalize_int(df.ingredients_n).value_counts(dropna=False)

<NA>    2631936
1        115960
4         49655
3         48606
5         44410
         ...   
227           1
291           1
294           1
302           1
258           1
Name: count, Length: 288, dtype: int64

In [23]:
# extract integer part of int and float
df.ingredients_n.astype(str).str.extract(r'(\d+)').astype('Int64').value_counts(dropna=False)

<NA>    2631936
1        115960
4         49655
3         48606
5         44410
         ...   
227           1
291           1
294           1
302           1
258           1
Name: count, Length: 288, dtype: int64

In [24]:
df.ingredients_n.value_counts(dropna=False)

ingredients_n
<NA>    2631936
1        115960
4         49655
3         48606
5         44410
         ...   
287           1
245           1
375           1
404           1
302           1
Name: count, Length: 288, dtype: Int64

In [25]:
df.known_ingredients_n.value_counts(dropna=False).head(30)

known_ingredients_n
<NA>    2632050
0         71038
3         40594
4         40043
6         33587
2         32349
5         29993
8         29194
7         28279
10        26749
9         26154
12        25461
14        25030
11        24653
13        23403
15        22469
16        22458
17        22287
18        21656
19        21422
20        20699
21        20202
22        19995
23        18994
24        18181
25        17597
26        17098
27        16739
28        16238
1         15517
Name: count, dtype: Int64

In [26]:
df.unknown_ingredients_n.value_counts(dropna=False).head(30)

unknown_ingredients_n
<NA>    2631319
0        381903
1        245219
2        130869
3         79207
4         50697
5         33486
6         23851
7         17724
8         13435
9         10712
10         8837
11         7172
12         6123
13         5314
14         4493
15         3957
16         3460
17         3051
18         2706
19         2366
20         2099
21         1936
22         1726
23         1544
24         1369
25         1333
26         1165
27         1150
28          988
Name: count, dtype: Int64

In [27]:
df.images.value_counts(dropna=False)

images
NaN                                                                                       2672290
["ingredients_fr"]                                                                         490960
["ingredients_en"]                                                                         164652
["ingredients_de"]                                                                          82222
["ingredients_es"]                                                                          64199
                                                                                           ...   
["ingredients_da","ingredients_en","ingredients_nb"]                                            1
["ingredients_de","ingredients_nb","ingredients_fr"]                                            1
["ingredients_fr","ingredients_sv","ingredients_da","ingredients_nb"]                           1
["ingredients_fr","ingredients_nl","ingredients_en","ingredients_el","ingredients_de"]          1
["ingredients

In [28]:
df.states.value_counts(dropna=False)

states
["en:ingredients-to-be-completed","en:ingredients-photo-to-be-selected"]    1799445
["en:ingredients-completed","en:ingredients-photo-selected"]                 588881
["en:ingredients-to-be-completed"]                                           467797
["en:ingredients-to-be-completed","en:ingredients-photo-selected"]           374151
["en:ingredients-completed"]                                                 289317
["en:ingredients-completed","en:ingredients-photo-to-be-selected"]           172328
Name: count, dtype: int64

In [29]:
df.quality_info.value_counts(dropna=False)

quality_info
NaN                                       2715272
["en:ingredients-percent-analysis-ok"]     976647
Name: count, dtype: int64

In [30]:
import numpy as np
df.quality_info.map(lambda x: np.nan if x =='[]' else x).value_counts(dropna=False)

quality_info
NaN                                       2715272
["en:ingredients-percent-analysis-ok"]     976647
Name: count, dtype: int64

## States

In [31]:
df.states.value_counts()

states
["en:ingredients-to-be-completed","en:ingredients-photo-to-be-selected"]    1799445
["en:ingredients-completed","en:ingredients-photo-selected"]                 588881
["en:ingredients-to-be-completed"]                                           467797
["en:ingredients-to-be-completed","en:ingredients-photo-selected"]           374151
["en:ingredients-completed"]                                                 289317
["en:ingredients-completed","en:ingredients-photo-to-be-selected"]           172328
Name: count, dtype: int64

In [32]:
df.states.explode().reset_index().pivot(index='index', columns=df.groupby(level=0).cumcount(), values='states')

KeyError: 0

In [ ]:
df_exploded = df.states.apply(lambda x: [s.replace("en:", "") for s in x]).explode().to_frame()
df_states = df_exploded.pivot_table(index=df_exploded.index, columns="states", aggfunc="size", fill_value=0)
df_states

In [ ]:
profile_states = ProfileReport(df_states, explorative=True)
profile_states.to_notebook_iframe()